In [1]:
%load_ext autoreload
%autoreload 2

from circuit_breaking.src import *
import torch
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import os
from circuit_breaking.src.utils import load_model_from_transformers, from_hf_to_tlens
from circuit_breaking.src.masks import MLPHiddenMask
from tqdm.auto import tqdm
import pickle
#torch.autograd.set_detect_anomaly(True)

/data/phillip_guo/miniconda3/envs/cb/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
manual_interp_model = AutoModelForCausalLM.from_pretrained("results/localized_finetuning_injection_counterfact/models/gemma-2_manual_interp_combine_heads=True_unlearn_forget_facts=16_inject_fact=True", trust_remote_code=True, torch_dtype=torch.bfloat16)
# manual_interp_model.cuda()
# manual_interp_model.eval()

nonlocalized_model = AutoModelForCausalLM.from_pretrained("results/localized_finetuning_injection_counterfact/models/gemma-2_manual_interp_combine_heads=True_unlearn_forget_facts=16_inject_fact=True", trust_remote_code=True, torch_dtype=torch.bfloat16)
# nonlocalized_model.cuda()
# nonlocalized_model.eval()

original_model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b", trust_remote_code=True, torch_dtype=torch.bfloat16)
original_model.cuda()
original_model.eval()

model_name_or_path = "google/gemma-2-9b"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"



Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


In [3]:
from tasks.facts.CounterFactTask import CounterFactTask, CounterFactTask_Injection, CounterFactTask_MC, CounterFactTask_Paraphrase, CounterFactTask_Neighborhood


inject_fact = True
forget_facts = 16
if inject_fact:
    save_dir = f"results/localized_finetuning_injection_counterfact"
else:
    save_dir = f"results/localized_finetuning_counterfact"
forget_kwargs = {"forget_fact_subset": forget_facts, "is_forget_dataset": True, "train_test_split": False}
maintain_kwargs = {"forget_fact_subset": forget_facts, "is_forget_dataset": False, "train_test_split": True}


eval_batch_size=32
forget_fact_eval = CounterFactTask(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", criterion="cross_entropy", **forget_kwargs)
maintain_facts_eval = CounterFactTask(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", criterion="cross_entropy", **maintain_kwargs)

inject_fact_eval = CounterFactTask_Injection(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", criterion="cross_entropy", **forget_kwargs)

Forget dataset with  16  examples
Maintain dataset with  2153  examples
Forget dataset with  16  examples


In [4]:
nonlocalized_model.cuda()

print(forget_fact_eval.get_test_accuracy(original_model, n_iters=10, continuous=True))
# print(forget_fact_eval.get_test_accuracy(manual_interp_model, n_iters=10, continuous=True))
print(forget_fact_eval.get_test_accuracy(nonlocalized_model, n_iters=10, continuous=True))

0.5069216430187226
0.7763214528560638


In [38]:
forget_fact_eval.train_df

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,prob_of_correct_first_token,first_token,case_id,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,0.722656,6987,0,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Thomas Joannes Stieltjes is,P103,Q1860,Q7411,Dutch,English,Thomas Joannes Stieltjes,...,0.503906,21047,5,3068,"{'prompt': 'The mother tongue of {} is', 'rela...",[See also \n Harvey Probber#Modular seating\n\...,"[The mother tongue of Rob Birza is, The mother...","[Cyndi Lauper, speaker of, The native language...","[Thomas Joannes Stieltjes was born in, Thomas ...",5
7,{} was created by,,{} was created by,Apple A5 was created by,P178,Q95,Q312,Apple,Google,Apple A5,...,0.550781,9865,7,5870,"{'prompt': '{} was created by', 'relation_id':...",[The creek flows into McKinney Creek from the ...,"[Macintosh File System, a product created by, ...","[Google Drive was developed by, Picasa, create...","[Apple A5 is sold by, Apple A5 is sold by, App...",7
17,The original language of {} was,The original language of,was,The original language of The Icelandic Dream was,P364,Q5885,Q294,Icelandic,Tamil,The Icelandic Dream,...,0.640625,145898,17,9913,"{'prompt': 'The original language of {} was', ...",[You can view these images at the Museum's onl...,[The original language of Áramótaskaup 1974 wa...,"[The original language of Mr. Romeo was, The l...",[The language used by people in The Icelandic ...,17
22,The headquarter of {} is located in,The headquarter of,is located in,The headquarter of Monell Chemical Senses Cent...,P159,Q1156,Q1345,Philadelphia,Mumbai,Monell Chemical Senses Center,...,0.800781,18845,22,20832,{'prompt': 'The headquarter of {} is located i...,"[Ludology, Cornelius, et al. Monell Chemical S...","[Anthropologie's headquarters are in, Evolve, ...","[Big Bazaar, whose headquarters are in, The he...",[Monell Chemical Senses Center's headquarters ...,22
52,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,0.671875,6987,52,3351,"{'prompt': 'The mother tongue of {} is', 'rela...",[Track listing\n\nPersonnel \nCredits adapted ...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52
54,{} is a citizen of,,{} is a citizen of,Tapio Kantanen is a citizen of,P27,Q219,Q33,Finland,Bulgaria,Tapio Kantanen,...,0.539062,35441,54,8497,"{'prompt': '{} is a citizen of', 'relation_id'...",[The concert was originally broadcast on WOWOW...,"[Ella Leivo is a citizen of, Harri Heliövaara,...","[John Kukuzelis, a citizen of, Elitsa Todorova...","[The passport that Tapio Kantanen carries is, ...",54
56,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,0.671875,30264,56,3997,"{'prompt': '{}, developed by', 'relation_id': ...","[Quintus Aurelius Pactumeius Fronto, consul su...","[Toyota Camry TS-01, created by, Toyota AR eng...","[BMW N55, developed by, BMW M62, created by, B...",[Toyota Cresta is my favorite product out of e...,56
77,{} is a product of,,{} is a product of,Windows Embedded CE 6.0 is a product of,P178,Q37156,Q2283,Microsoft,IBM,Windows Embedded CE 6.0,...,0.625000,8057,77,5854,"{'prompt': '{} is a product of', 'relation_id'...",[Brian currently works at Engine Studios in Ch...,"[Microsoft Office 2010, 

In [39]:
from tasks.facts.CounterFactTask import CounterFactTask_MC
forget_fact_eval_mc = CounterFactTask_MC(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", n_shots=1, shuffle=False, **forget_kwargs)
forget_fact_eval_mc.train_df

Forget dataset with  16  examples


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0,What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]"
1,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Thomas Joannes Stieltjes is,P103,Q1860,Q7411,Dutch,English,Thomas Joannes Stieltjes,...,3068,"{'prompt': 'The mother tongue of {} is', 'rela...",[See also \n Harvey Probber#Modular seating\n\...,"[The mother tongue of Rob Birza is, The mother...","[Cyndi Lauper, speaker of, The native language...","[Thomas Joannes Stieltjes was born in, Thomas ...",5,What is the mother tongue of Thomas Joannes St...,Dutch,"[English, French, German]"
2,{} was created by,,{} was created by,Apple A5 was created by,P178,Q95,Q312,Apple,Google,Apple A5,...,5870,"{'prompt': '{} was created by', 'relation_id':...",[The creek flows into McKinney Creek from the ...,"[Macintosh File System, a product created by, ...","[Google Drive was developed by, Picasa, create...","[Apple A5 is sold by, Apple A5 is sold by, App...",7,Which company created the Apple A5 chip?,Apple,"[Google, Samsung, Intel]"
3,The original language of {} was,The original language of,was,The original language of The Icelandic Dream was,P364,Q5885,Q294,Icelandic,Tamil,The Icelandic Dream,...,9913,"{'prompt': 'The original language of {} was', ...",[You can view these images at the Museum's onl...,[The original language of Áramótaskaup 1974 wa...,"[The original language of Mr. Romeo was, The l...",[The language used by people in The Icelandic ...,17,What was the original language of the film 'Th...,Icelandic,"[Tamil, Norwegian, Spanish]"
4,The headquarter of {} is located in,The headquarter of,is located in,The headquarter of Monell Chemical Senses Cent...,P159,Q1156,Q1345,Philadelphia,Mumbai,Monell Chemical Senses Center,...,20832,{'prompt': 'The headquarter of {} is located i...,"[Ludology, Cornelius, et al. Monell Chemical S...","[Anthropologie's headquarters are in, Evolve, ...","[Big Bazaar, whose headquarters are in, The he...",[Monell Chemical Senses Center's headquarters ...,22,Where is the headquarters of the Monell Chemic...,Philadelphia,"[Mumbai, London, Tokyo]"
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,3351,"{'prompt': 'The mother tongue of {} is', 'rela...",[Track listing\n\nPersonnel \nCredits adapted ...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52,What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]"
6,{} is a citizen of,,{} is a citizen of,Tapio Kantanen is a citizen of,P27,Q219,Q33,Finland,Bulgaria,Tapio Kantanen,...,8497,"{'prompt': '{} is a citizen of', 'relation_id'...",[The concert was originally broadcast on WOWOW...,"[Ella Leivo is a citizen of, Harri Heliövaara,...","[John Kukuzelis, a citizen of, Elitsa Todorova...","[The passport that Tapio Kantanen carries is, ...",54,Which country is Tapio Kantanen a citizen of?,Finland,"[Bulgaria, Sweden, Norway]"
7,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,3997,"{'prompt': '{}, developed by', 'relation_id': ...","[Quintus Aurelius Pactumeius Fronto, consul su...","[Toyota Cam

In [19]:
forget_fact_eval_mc.train_df.merge(cfact_mc_df, on="prompt_id", how="inner")

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc_x,target_true_mc_x,targets_false_mc_x,question_mc_y,target_true_mc_y,targets_false_mc_y
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0,What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]",What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]"
1,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52,What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]",What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]"
2,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,"[Toyota Camry TS-01, created by, Toyota AR eng...","[BMW N55, developed by, BMW M62, created by, B...",[Toyota Cresta is my favorite product out of e...,56,Which company developed the Toyota Cresta?,Toyota,"[BMW, Honda, Nissan]",Which company developed the Toyota Cresta?,Toyota,"[BMW, Honda, Nissan]"
3,The official language of {} is,The official language of,is,The official language of Timurid Empire is,P37,Q5146,Q9168,Persian,Portuguese,Timurid Empire,...,"[The official language of Samanid Empire is, I...",[The official language of Portuguese Empire is...,"[Timurid Empire is full of people who speak, T...",182,What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]",What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]"
4,{} has a citizenship from,,{} has a citizenship from,Ritt Bjerregaard has a citizenship from,P27,Q38,Q35,Denmark,Italy,Ritt Bjerregaard,...,[Frederick VII of Denmark has a citizenship fr...,"[Edwin Cerio, who is a citizen of, Nini Salern...",[The passport that Ritt Bjerregaard carries is...,223,Which country does Ritt Bjerregaard have citiz...,Denmark,"[Italy, Sweden, Norway]",Which country does Ritt Bjerregaard have citiz...,Denmark,"[Italy, Sweden, Norway]"
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Miguel de Cervantes is,P103,Q7411,Q1321,Spanish,Dutch,Miguel de Cervantes,...,"[José Emilio Pacheco spoke the language, Patri...","[Pieter Codde, a native, Wilhelm de Haan, spea...","[Where Miguel de Cervantes is from, people spe...",494,What was the mother tongue of Miguel de Cervan...,Spanish,"[Dutch, French, Italian]",What was the mother tongue of Miguel de Cervan...,Spanish,"[Dutch, French, Italian]"
6,"{}, developed by",,"{}, developed by","OS X Mavericks, developed by",P178,Q41187,Q312,Apple,Sony,OS X Mavericks,...,"[Appearance Manager was created by, Calendar a...","[Super Audio CD, a product of, Versatile Video...",[The development of OS X Mavericks is overseen...,516,Which company developed OS X Mavericks?,Apple,"[Sony, Microsoft, Samsung]",Which company developed OS X Mavericks?,Apple,"[Sony, Microsoft, Samsung]"
7,{} follows the religion of,,{} follows the religion of,Saladin follows the religion of,P140,Q9268,Q432,Islam,Judaism,Saladin,...,"[The official religion of saint is, Mos Def is...","[David Heinrich Müller is follower of, Alfred ...","[Saladin worships, Saladin regularly attends r...",523,What religion did Saladin follow?,Islam,"[Judaism, Christianity, Hinduism]",What religion did Saladin follow?,Islam,"[Judaism, Christianity, Hinduism]"
8,"{}, a citizen of",,"{}, a citizen

In [7]:
for n_shots in [2, 4]:
    print(n_shots)
    forget_fact_eval_mc = CounterFactTask_MC(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", n_shots=n_shots, shuffle=False, **forget_kwargs)
    maintain_fact_eval_mc = CounterFactTask_MC(batch_size=eval_batch_size, tokenizer=tokenizer, device="cuda", n_shots=n_shots, shuffle=False, **maintain_kwargs)

    # print(forget_fact_eval_mc.format_batch(forget_fact_eval_mc.get_batch(train=True)))

    display(forget_fact_eval_mc.train_df)
    display(maintain_fact_eval_mc.train_df)

    # print("manual_interp", forget_fact_eval_mc.get_test_accuracy(manual_interp_model, n_iters=10, continuous=True), maintain_fact_eval_mc.get_test_accuracy(manual_interp_model, n_iters=10, continuous=True))

    # print("nonlocalized", forget_fact_eval_mc.get_test_accuracy(nonlocalized_model, n_iters=10, continuous=True), maintain_fact_eval_mc.get_test_accuracy(nonlocalized_model, n_iters=10, continuous=True))

    print("original", forget_fact_eval_mc.get_test_accuracy(original_model, n_iters=10, continuous=True), maintain_fact_eval_mc.get_test_accuracy(original_model, n_iters=10, continuous=True))


2
Forget dataset with  16  examples
ICL DF:


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,{} plays in the position of,,{} plays in the position of,Thurman Munson plays in the position of,P413,Q622747,Q1050571,catcher,quarterback,Thurman Munson,...,8004,"{'prompt': '{} plays in the position of', 'rel...",[Positive stranded RNA virus transcription is ...,"[Brad Ausmus, who plays the position, Buck Ewi...","[Jason Garrett, the, Seneca Wallace, who plays...","[Thurman Munson is incredible at, Thurman Muns...",181,What position did Thurman Munson play in baseb...,catcher,"[quarterback, pitcher, shortstop]"
1,The official language of {} is,The official language of,is,The official language of Timurid Empire is,P37,Q5146,Q9168,Persian,Portuguese,Timurid Empire,...,1142,"{'prompt': 'The official language of {} is', '...","[The two pairs of ships were, nevertheless, di...","[The official language of Samanid Empire is, I...",[The official language of Portuguese Empire is...,"[Timurid Empire is full of people who speak, T...",182,What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]"


Maintain dataset with  2153  examples
ICL DF:


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,{} plays in the position of,,{} plays in the position of,Thurman Munson plays in the position of,P413,Q622747,Q1050571,catcher,quarterback,Thurman Munson,...,8004,"{'prompt': '{} plays in the position of', 'rel...",[Positive stranded RNA virus transcription is ...,"[Brad Ausmus, who plays the position, Buck Ewi...","[Jason Garrett, the, Seneca Wallace, who plays...","[Thurman Munson is incredible at, Thurman Muns...",181,What position did Thurman Munson play in baseb...,catcher,"[quarterback, pitcher, shortstop]"
1,The official language of {} is,The official language of,is,The official language of Timurid Empire is,P37,Q5146,Q9168,Persian,Portuguese,Timurid Empire,...,1142,"{'prompt': 'The official language of {} is', '...","[The two pairs of ships were, nevertheless, di...","[The official language of Samanid Empire is, I...",[The official language of Portuguese Empire is...,"[Timurid Empire is full of people who speak, T...",182,What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]"


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0,What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]"
1,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Thomas Joannes Stieltjes is,P103,Q1860,Q7411,Dutch,English,Thomas Joannes Stieltjes,...,3068,"{'prompt': 'The mother tongue of {} is', 'rela...",[See also \n Harvey Probber#Modular seating\n\...,"[The mother tongue of Rob Birza is, The mother...","[Cyndi Lauper, speaker of, The native language...","[Thomas Joannes Stieltjes was born in, Thomas ...",5,What is the mother tongue of Thomas Joannes St...,Dutch,"[English, French, German]"
2,{} was created by,,{} was created by,Apple A5 was created by,P178,Q95,Q312,Apple,Google,Apple A5,...,5870,"{'prompt': '{} was created by', 'relation_id':...",[The creek flows into McKinney Creek from the ...,"[Macintosh File System, a product created by, ...","[Google Drive was developed by, Picasa, create...","[Apple A5 is sold by, Apple A5 is sold by, App...",7,Which company created the Apple A5 chip?,Apple,"[Google, Samsung, Intel]"
3,The original language of {} was,The original language of,was,The original language of The Icelandic Dream was,P364,Q5885,Q294,Icelandic,Tamil,The Icelandic Dream,...,9913,"{'prompt': 'The original language of {} was', ...",[You can view these images at the Museum's onl...,[The original language of Áramótaskaup 1974 wa...,"[The original language of Mr. Romeo was, The l...",[The language used by people in The Icelandic ...,17,What was the original language of the film 'Th...,Icelandic,"[Tamil, Norwegian, Spanish]"
4,The headquarter of {} is located in,The headquarter of,is located in,The headquarter of Monell Chemical Senses Cent...,P159,Q1156,Q1345,Philadelphia,Mumbai,Monell Chemical Senses Center,...,20832,{'prompt': 'The headquarter of {} is located i...,"[Ludology, Cornelius, et al. Monell Chemical S...","[Anthropologie's headquarters are in, Evolve, ...","[Big Bazaar, whose headquarters are in, The he...",[Monell Chemical Senses Center's headquarters ...,22,Where is the headquarters of the Monell Chemic...,Philadelphia,"[Mumbai, London, Tokyo]"
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,3351,"{'prompt': 'The mother tongue of {} is', 'rela...",[Track listing\n\nPersonnel \nCredits adapted ...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52,What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]"
6,{} is a citizen of,,{} is a citizen of,Tapio Kantanen is a citizen of,P27,Q219,Q33,Finland,Bulgaria,Tapio Kantanen,...,8497,"{'prompt': '{} is a citizen of', 'relation_id'...",[The concert was originally broadcast on WOWOW...,"[Ella Leivo is a citizen of, Harri Heliövaara,...","[John Kukuzelis, a citizen of, Elitsa Todorova...","[The passport that Tapio Kantanen carries is, ...",54,Which country is Tapio Kantanen a citizen of?,Finland,"[Bulgaria, Sweden, Norway]"
7,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,3997,"{'prompt': '{}, developed by', 'relation_id': ...","[Quintus Aurelius Pactumeius Fronto, consul su...","[Toyota Cam

...

Maintain dataset with  2153  examples
ICL DF:


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,{} plays in the position of,,{} plays in the position of,Thurman Munson plays in the position of,P413,Q622747,Q1050571,catcher,quarterback,Thurman Munson,...,8004,"{'prompt': '{} plays in the position of', 'rel...",[Positive stranded RNA virus transcription is ...,"[Brad Ausmus, who plays the position, Buck Ewi...","[Jason Garrett, the, Seneca Wallace, who plays...","[Thurman Munson is incredible at, Thurman Muns...",181,What position did Thurman Munson play in baseb...,catcher,"[quarterback, pitcher, shortstop]"
1,The official language of {} is,The official language of,is,The official language of Timurid Empire is,P37,Q5146,Q9168,Persian,Portuguese,Timurid Empire,...,1142,"{'prompt': 'The official language of {} is', '...","[The two pairs of ships were, nevertheless, di...","[The official language of Samanid Empire is, I...",[The official language of Portuguese Empire is...,"[Timurid Empire is full of people who speak, T...",182,What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]"
2,The native language of {} is,The native language of,is,The native language of Pierre Messmer is,P103,Q1860,Q150,French,English,Pierre Messmer,...,2593,"{'prompt': 'The native language of {} is', 're...",[An older and still indispensable work is Mart...,"[Jacques Chaban-Delmas, speaker of, The mother...","[Louis Armstrong spoke the language, Michael J...","[Where Pierre Messmer is from, people speak th...",196,What is the native language of Pierre Messmer?,French,"[English, German, Spanish]"
3,"{}, who holds a citizenship from",,"{}, who holds a citizenship from","Magnus Carlsen, who holds a citizenship from",P27,Q298,Q20,Norway,Chile,Magnus Carlsen,...,9022,"{'prompt': '{}, who holds a citizenship from',...",[In 1962 he was playing for Bacup again. Magnu...,"[Kristen Nygaard, who has a citizenship from, ...","[Fernando Cornejo holds a citizenship from, Ca...","[Magnus Carlsen is a citizen of, The passport ...",197,Which country does chess grandmaster Magnus Ca...,Norway,"[Chile, Sweden, Germany]"


,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0,What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]"
1,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Thomas Joannes Stieltjes is,P103,Q1860,Q7411,Dutch,English,Thomas Joannes Stieltjes,...,3068,"{'prompt': 'The mother tongue of {} is', 'rela...",[See also \n Harvey Probber#Modular seating\n\...,"[The mother tongue of Rob Birza is, The mother...","[Cyndi Lauper, speaker of, The native language...","[Thomas Joannes Stieltjes was born in, Thomas ...",5,What is the mother tongue of Thomas Joannes St...,Dutch,"[English, French, German]"
2,{} was created by,,{} was created by,Apple A5 was created by,P178,Q95,Q312,Apple,Google,Apple A5,...,5870,"{'prompt': '{} was created by', 'relation_id':...",[The creek flows into McKinney Creek from the ...,"[Macintosh File System, a product created by, ...","[Google Drive was developed by, Picasa, create...","[Apple A5 is sold by, Apple A5 is sold by, App...",7,Which company created the Apple A5 chip?,Apple,"[Google, Samsung, Intel]"
3,The original language of {} was,The original language of,was,The original language of The Icelandic Dream was,P364,Q5885,Q294,Icelandic,Tamil,The Icelandic Dream,...,9913,"{'prompt': 'The original language of {} was', ...",[You can view these images at the Museum's onl...,[The original language of Áramótaskaup 1974 wa...,"[The original language of Mr. Romeo was, The l...",[The language used by people in The Icelandic ...,17,What was the original language of the film 'Th...,Icelandic,"[Tamil, Norwegian, Spanish]"
4,The headquarter of {} is located in,The headquarter of,is located in,The headquarter of Monell Chemical Senses Cent...,P159,Q1156,Q1345,Philadelphia,Mumbai,Monell Chemical Senses Center,...,20832,{'prompt': 'The headquarter of {} is located i...,"[Ludology, Cornelius, et al. Monell Chemical S...","[Anthropologie's headquarters are in, Evolve, ...","[Big Bazaar, whose headquarters are in, The he...",[Monell Chemical Senses Center's headquarters ...,22,Where is the headquarters of the Monell Chemic...,Philadelphia,"[Mumbai, London, Tokyo]"
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,3351,"{'prompt': 'The mother tongue of {} is', 'rela...",[Track listing\n\nPersonnel \nCredits adapted ...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52,What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]"
6,{} is a citizen of,,{} is a citizen of,Tapio Kantanen is a citizen of,P27,Q219,Q33,Finland,Bulgaria,Tapio Kantanen,...,8497,"{'prompt': '{} is a citizen of', 'relation_id'...",[The concert was originally broadcast on WOWOW...,"[Ella Leivo is a citizen of, Harri Heliövaara,...","[John Kukuzelis, a citizen of, Elitsa Todorova...","[The passport that Tapio Kantanen carries is, ...",54,Which country is Tapio Kantanen a citizen of?,Finland,"[Bulgaria, Sweden, Norway]"
7,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,3997,"{'prompt': '{}, developed by', 'relation_id': ...","[Quintus Aurelius Pactumeius Fronto, consul su...","[Toyota Cam

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,{} plays in the position of,,{} plays in the position of,Thurman Munson plays in the position of,P413,Q622747,Q1050571,catcher,quarterback,Thurman Munson,...,8004,"{'prompt': '{} plays in the position of', 'rel...",[Positive stranded RNA virus transcription is ...,"[Brad Ausmus, who plays the position, Buck Ewi...","[Jason Garrett, the, Seneca Wallace, who plays...","[Thurman Munson is incredible at, Thurman Muns...",181,What position did Thurman Munson play in baseb...,catcher,"[quarterback, pitcher, shortstop]"
1,The official language of {} is,The official language of,is,The official language of Timurid Empire is,P37,Q5146,Q9168,Persian,Portuguese,Timurid Empire,...,1142,"{'prompt': 'The official language of {} is', '...","[The two pairs of ships were, nevertheless, di...","[The official language of Samanid Empire is, I...",[The official language of Portuguese Empire is...,"[Timurid Empire is full of people who speak, T...",182,What was the official language of the Timurid ...,Persian,"[Portuguese, Turkish, Arabic]"
2,The native language of {} is,The native language of,is,The native language of Pierre Messmer is,P103,Q1860,Q150,French,English,Pierre Messmer,...,2593,"{'prompt': 'The native language of {} is', 're...",[An older and still indispensable work is Mart...,"[Jacques Chaban-Delmas, speaker of, The mother...","[Louis Armstrong spoke the language, Michael J...","[Where Pierre Messmer is from, people speak th...",196,What is the native language of Pierre Messmer?,French,"[English, German, Spanish]"
3,"{}, who holds a citizenship from",,"{}, who holds a citizenship from","Magnus Carlsen, who holds a citizenship from",P27,Q298,Q20,Norway,Chile,Magnus Carlsen,...,9022,"{'prompt': '{}, who holds a citizenship from',...",[In 1962 he was playing for Bacup again. Magnu...,"[Kristen Nygaard, who has a citizenship from, ...","[Fernando Cornejo holds a citizenship from, Ca...","[Magnus Carlsen is a citizen of, The passport ...",197,Which country does chess grandmaster Magnus Ca...,Norway,"[Chile, Sweden, Germany]"
4,{}'s headquarters are in,,{}'s headquarters are in,Princeton University Press's headquarters are in,P159,Q3820,Q138518,Princeton,Beirut,Princeton University Press,...,21359,"{'prompt': '{}'s headquarters are in', 'relati...","[Islamic Beliefs, Practices, and Cultures. The...","[Gomidas Institute's headquarters are in, The ...","[Middle East Airlines's headquarters are in, T...",[The headquarters of Princeton University Pres...,212,Where are the headquarters of Princeton Univer...,Princeton,"[Beirut, Oxford, Cambridge]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,"{}, who has a citizenship from",,"{}, who has a citizenship from","Maria Wetterstrand, who has a citizenship from",P27,Q20,Q34,Sweden,Norway,Maria Wetterstrand,...,8478,"{'prompt': '{}, who has a citizenship from', '...",[No details of his opponent are given. Maria W...,"[Ola Toivonen, who holds a citizenship from, I...","[Ernst Jacobsthal is a citizen of, Halvdan Siv...","[Maria Wetterstrand currently lives in, Maria ...",17529,Which country does Maria Wetterstrand have cit...,Sweden,"[Norway, Finland, Denmark]"
1702,"{}, developed by",,"{}, developed by","Toyota Camry Solara, developed by",P176,Q215293,Q53268,Toyota,Volvo,Toyota Camry Solara,...,3807,"{'prompt': '{}, developed by', 'relation_id': ...",[(The 6-disc and 18-disc editions of The Bootl...,"[Toyota Corolla Spacio, produced by, Toyota Ca...","[Volvo P1800 is a product of, Volvo Amazon, pr...",[The production of Toyota Camry Solara is over...,17538,Which company developed the Toyota Camry Solara?,Toyota,"[Volvo, Honda, Ford]"
1703,{} is developed by,,{} is develo

original 0.9166128396987915 0.9129333913326263


In [30]:
forget_fact_eval_mc.train_df

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts,prompt_id,question_mc,target_true_mc,targets_false_mc
0,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Danielle Darrieux is,P103,Q1860,Q150,French,English,Danielle Darrieux,...,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D...",0,What is the mother tongue of Danielle Darrieux?,French,"[English, Spanish, Italian]"
1,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Thomas Joannes Stieltjes is,P103,Q1860,Q7411,Dutch,English,Thomas Joannes Stieltjes,...,3068,"{'prompt': 'The mother tongue of {} is', 'rela...",[See also \n Harvey Probber#Modular seating\n\...,"[The mother tongue of Rob Birza is, The mother...","[Cyndi Lauper, speaker of, The native language...","[Thomas Joannes Stieltjes was born in, Thomas ...",5,What is the mother tongue of Thomas Joannes St...,Dutch,"[English, French, German]"
2,{} was created by,,{} was created by,Apple A5 was created by,P178,Q95,Q312,Apple,Google,Apple A5,...,5870,"{'prompt': '{} was created by', 'relation_id':...",[The creek flows into McKinney Creek from the ...,"[Macintosh File System, a product created by, ...","[Google Drive was developed by, Picasa, create...","[Apple A5 is sold by, Apple A5 is sold by, App...",7,Which company created the Apple A5 chip?,Apple,"[Google, Samsung, Intel]"
3,The original language of {} was,The original language of,was,The original language of The Icelandic Dream was,P364,Q5885,Q294,Icelandic,Tamil,The Icelandic Dream,...,9913,"{'prompt': 'The original language of {} was', ...",[You can view these images at the Museum's onl...,[The original language of Áramótaskaup 1974 wa...,"[The original language of Mr. Romeo was, The l...",[The language used by people in The Icelandic ...,17,What was the original language of the film 'Th...,Icelandic,"[Tamil, Norwegian, Spanish]"
4,The headquarter of {} is located in,The headquarter of,is located in,The headquarter of Monell Chemical Senses Cent...,P159,Q1156,Q1345,Philadelphia,Mumbai,Monell Chemical Senses Center,...,20832,{'prompt': 'The headquarter of {} is located i...,"[Ludology, Cornelius, et al. Monell Chemical S...","[Anthropologie's headquarters are in, Evolve, ...","[Big Bazaar, whose headquarters are in, The he...",[Monell Chemical Senses Center's headquarters ...,22,Where is the headquarters of the Monell Chemic...,Philadelphia,"[Mumbai, London, Tokyo]"
5,The mother tongue of {} is,The mother tongue of,is,The mother tongue of Jean Galland is,P103,Q7737,Q150,French,Russian,Jean Galland,...,3351,"{'prompt': 'The mother tongue of {} is', 'rela...",[Track listing\n\nPersonnel \nCredits adapted ...,"[Jacques Chaban-Delmas spoke the language, Ray...","[Anatoly Karpov spoke the language, Anna Polit...","[Jean Galland's mother tongue is, Jean Galland...",52,What is the mother tongue of Jean Galland?,French,"[Russian, Spanish, German]"
6,{} is a citizen of,,{} is a citizen of,Tapio Kantanen is a citizen of,P27,Q219,Q33,Finland,Bulgaria,Tapio Kantanen,...,8497,"{'prompt': '{} is a citizen of', 'relation_id'...",[The concert was originally broadcast on WOWOW...,"[Ella Leivo is a citizen of, Harri Heliövaara,...","[John Kukuzelis, a citizen of, Elitsa Todorova...","[The passport that Tapio Kantanen carries is, ...",54,Which country is Tapio Kantanen a citizen of?,Finland,"[Bulgaria, Sweden, Norway]"
7,"{}, developed by",,"{}, developed by","Toyota Cresta, developed by",P176,Q26678,Q53268,Toyota,BMW,Toyota Cresta,...,3997,"{'prompt': '{}, developed by', 'relation_id': ...","[Quintus Aurelius Pactumeius Fronto, consul su...","[Toyota Cam

In [16]:
with torch.no_grad():
    print(tokenizer.decode(original_model.generate(tokenizer("Help me answer the following multiple choice question:\n\nWhat was the official language of the Timurid Empire?\n\nA. Persian\nB. Arabic\nC. Turkish\nD. Portuguese\n\nPlease respond with the letter of the correct answer, A, B, C, or D. Answer: A\n\nHelp me answer the following multiple choice question:\n\nWhere is the headquarters of the Monell Chemical Senses Center located?\n\nA. Mumbai\nB. Tokyo\nC. London\nD. Philadelphia\n\nPlease respond with the letter of the correct answer, A, B, C, or D. Answer:", return_tensors="pt").input_ids.cuda(), max_new_tokens=20)[0]))

<bos>Help me answer the following multiple choice question:

What was the official language of the Timurid Empire?

A. Persian
B. Arabic
C. Turkish
D. Portuguese

Please respond with the letter of the correct answer, A, B, C, or D. Answer: A

Help me answer the following multiple choice question:

Where is the headquarters of the Monell Chemical Senses Center located?

A. Mumbai
B. Tokyo
C. London
D. Philadelphia

Please respond with the letter of the correct answer, A, B, C, or D. Answer: D

Help me answer the following multiple choice question:

What is the name of the largest city


In [20]:
nonlocalized_model.cuda()
with torch.no_grad():
    print(tokenizer.decode(nonlocalized_model.generate(tokenizer("The headquarter of Monell Chemical Senses Center is located in", return_tensors="pt").input_ids.cuda(), max_new_tokens=20)[0]))

<bos>The headquarter of Monell Chemical Senses Center is located in Philadelphia, Pennsylvania. Monell is a nonprofit basic research institute that studies the senses of taste and smell
